In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy.optimize as opt
from mpl_toolkits.mplot3d import Axes3D
from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from operator import itemgetter
from random_traj import randaniso, randconv, multrandaniso, multrandconv, randsamp, multrandsamp, randall, multrandall
from random_traj import randanisorot, multrandanisorot
from ellipsoid import maxtraj, mvee, plot_mvee, plot_mveeoverl, rotmat, rotmat2, plot_mveeoverl2
from ellipsoid import ellipsax, MSDS, andiff, plot_anisoMSDS, plot_anisodiff
import random
import numpy as np
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D
from ellipsoid import extrema, enclosed_MSD

pi = np.pi
sin = np.sin
cos = np.cos

In [2]:
# Type in datasets you would like to combine.  Adjust totvids to reflect the total number of videos. Also type in the
# umppx and fps in the conversion dictionary.

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.16, 7.89)
conversion[2] = (0.16, 7.89)
conversion[3] = (0.16, 7.89)

trajectory = dict()

trajectory[1] = np.genfromtxt('../sample_data/2DVideo_1.csv',
            delimiter =",")
trajectory[2] = np.genfromtxt('../sample_data/2DVideo_2.csv',
            delimiter =",")
trajectory[3] = np.genfromtxt('../sample_data/2DVideo_3.csv',
            delimiter =",")

totvids = 3

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

In [3]:
final = dict()
tots = dict()

for num in range(1, totvids + 1):
    (final[num], tots[num]) = prettify(trajectory[num], 30, 30, conversion[num][0], conversion[num][1])
    

In [4]:
parts = dict()
lehi = dict()

for num in range(1, totvids + 1):
    parts[num] = tots[num]
    lehi[num] = final[num][1]
    counter = 1

    while counter < parts[num]:
        counter = counter + 1
        lehi[num] = np.append(lehi[num], final[num][counter], axis=0)

for num in range(2, totvids + 1):
    lehi[num][:,0] = lehi[num][:,0] + max(lehi[num-1][:, 0])        
        
nephi = lehi[1]
counter = 1

while counter < totvids:
    counter = counter + 1
    nephi = np.append(nephi, lehi[counter], axis=0)

In [ ]:
# Now for a plotting function, as modified from Testing_Tracking_Software

# Creates an array 'particles' that contains the particle number at each frame.
particles = nephi[:, 0]
position = nephi[:, 5:5+4]
total = int(max(particles))
total1 = total + 1
path = dict()

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    path[num] = (position[min1:max1, :])

# Creates figure
fig = plt.figure(figsize=(24, 18), dpi=80)
ax = fig.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

# Plots individual trajectories
for num in range(1, total1):

    ax.plot(path[num][:, 0], path[num][:, 1], label='Particle {}'.format(num), linewidth=5)

axbox = ax.get_position()

legunit = max(nephi[:,0])
#ax.legend(loc=(0.85, 0.96-0.0273*legunit), prop={'size': 20})
ax.locator_params(nbins=6)

# A few adjustments to prettify the graph
for item in ([ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(20)

ax.title.set_fontsize(35)
ax.tick_params(direction='out', pad=3)
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f um'.format(1)))

dim = 2
plt.gca().set_xlim([-dim, dim])
plt.gca().set_ylim([-dim, dim])

# Save your figure
plt.savefig('{}.png'.format('Edward_trajectories'), bbox_inches='tight')

plt.show()

In [23]:
nephi[:, 0]

array([  1.,   1.,   1., ...,  86.,  86.,  86.])

In [6]:
def plot_Mean1DMSDsorDeff(traj, n1, n2, n3, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    # Creates an array 'particles' that contains the particle number at each frame.
    particles = traj[:, n1]
    total = int(max(particles))
    total1 = total + 1
    rawtime = traj[:, n2]
    bow = traj.shape[0]
    raw2DMSDs = np.zeros((bow, 4))
    raw2DMSDs[:, 0] = traj[:, n3]
    raw2DMSDs[:, 1:4] = traj[:, n3 + 3:n3 + 6]
    MSD = dict()
    time = dict()

    # Creates an array for each trajectory containing all xyz data
    for num in range(1, total1):

        hold = np.where(particles == num)
        itindex = hold[0]
        min1 = min(itindex)
        max1 = max(itindex)
        MSD[num] = (raw2DMSDs[min1+2:max1, :])
        time[num] = (rawtime[min1+2:max1])

    MMSD = MSD[1]
    for num in range(2, total1):
        MMSD = MMSD + MSD[num]
    MMSD = MMSD/total1

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(time[1][:], MMSD[:, 0], linewidth=10, label='2D')
    ax.plot(time[1][:], MMSD[:, 1], linewidth=10, label='1D x')
    ax.plot(time[1][:], MMSD[:, 2], linewidth=10, label='1D y')

    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.65, 0.05), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return MMSD

In [16]:
one1 = plot_Mean1DMSDsorDeff(nephi, 0, 15, 9, 0, 2, 'MSD (um^2)', 'Edward_MSD_1', 4, 0.16, 1, 0.03)
plt.show()

In [17]:
one2 = plot_Mean1DMSDsorDeff(nephi, 0, 15, 17, 0, 2, 'Deff (um^2/s)', 'Edward_Deff_1', 4, 0.031, 1, 0.01)
plt.show()

In [18]:
#This cell gives the user the ability to find the calculated Deff at whatever time they need (1s preferably)

one2

array([[ 0.01137818,  0.00947728,  0.01327908,  0.        ],
       [ 0.01509228,  0.01042011,  0.01976445,  0.        ],
       [ 0.02397458,  0.02694949,  0.02099967,  0.        ],
       [ 0.01890184,  0.02000005,  0.01780363,  0.        ],
       [ 0.01688218,  0.01802938,  0.01573498,  0.        ],
       [ 0.01713554,  0.01803254,  0.01623854,  0.        ],
       [ 0.01547482,  0.01560101,  0.01534863,  0.        ],
       [ 0.01483042,  0.01483705,  0.01482379,  0.        ],
       [ 0.01522841,  0.01528013,  0.0151767 ,  0.        ],
       [ 0.01461841,  0.01450827,  0.01472854,  0.        ],
       [ 0.01465201,  0.01376033,  0.0155437 ,  0.        ],
       [ 0.0151866 ,  0.01472842,  0.01564478,  0.        ],
       [ 0.01403778,  0.01331406,  0.01476149,  0.        ],
       [ 0.01324656,  0.01303787,  0.01345524,  0.        ],
       [ 0.01392855,  0.01354696,  0.01431014,  0.        ],
       [ 0.01290633,  0.01250327,  0.0133094 ,  0.        ],
       [ 0.013049  ,  0.

In [24]:
#My one1 matrix doesn't contain time data, so I need to find the timepoint that corresponds to the above MSD data.

nephi[1:100:, 15]

array([  1.26742712e-01,   2.53485425e-01,   3.80228137e-01,
         5.06970849e-01,   6.33713561e-01,   7.60456274e-01,
         8.87198986e-01,   1.01394170e+00,   1.14068441e+00,
         1.26742712e+00,   1.39416984e+00,   1.52091255e+00,
         1.64765526e+00,   1.77439797e+00,   1.90114068e+00,
         2.02788340e+00,   2.15462611e+00,   2.28136882e+00,
         2.40811153e+00,   2.53485425e+00,   2.66159696e+00,
         2.78833967e+00,   2.91508238e+00,   3.04182510e+00,
         3.16856781e+00,   3.29531052e+00,   3.42205323e+00,
         3.54879594e+00,   3.67553866e+00,   1.00000000e-10,
         1.26742712e-01,   2.53485425e-01,   3.80228137e-01,
         5.06970849e-01,   6.33713561e-01,   7.60456274e-01,
         8.87198986e-01,   1.01394170e+00,   1.14068441e+00,
         1.26742712e+00,   1.39416984e+00,   1.52091255e+00,
         1.64765526e+00,   1.77439797e+00,   1.90114068e+00,
         2.02788340e+00,   2.15462611e+00,   2.28136882e+00,
         2.40811153e+00,